# Asymmetric Key Encryption and Digital Signature Verification using OpenSSL 

ในการเข้ารหัสข้อมูลด้วยกุญแจเข้ารหัส จะมีวิธีการเข้ารหัสอยู่สองชนิด คือ การเข้ารหัสแบบสมมาตร (symmetric key encryption) และการเข้ารหัสแบบอสมมาตร (asymmetric key encryption) โดยการเข้ารหัสแบบสมมาตรจะใช้กุญแจลับ (secret key) อันเดียวเพื่อเข้ารหัสและถอดรหัส โดยในการรับส่งข้อมูลผ่านตัวกลางนั้น secret key จะต้องถูกส่งไปหาผู้รับด้วยเพื่อใช้ในการถอดรหัส ซึ่งอาจจะถูกดักฟังโดยผู้ไม่ประสงค์ดีได้ ดังนนั้นจึงมีการพัฒนาวิธีการเข้ารหัสแบบอสมมาตรขึ้นมาเพื่อแก้ปัญหานี้ โดยใช้กุญแจสองตัว คือ กุญแจสาธารณะ (public key) และกุญแจส่วนตัว (private key) ในการเข้ารหัสและถอดรหัส ซึ่งจะต่างจากการเข้ารหัสแบบสมมาตรที่ การเข้ารหัสจะใช้กุญแจอันหนึ่ง และต้องใช้กุญแจอีกอันเพื่อถอดรหัสเท่านั้น ในทางปฏิบัติ มักจะใช้ public key เพื่อเข้ารหัสและใช้ private key เพื่อถอดรหัส โดย private key จะถูกสุ่มขึ้นมาที่ผู้รับสารและผู้รับสารจะสร้างคู่ public key ขึ้นมาและกระจาย key สู่สารธารณะเพื่อให้ผู้ส่งสารสามารถนำไปเข้ารหัสได้

อย่างไรก็ตาม ในการเข้ารหัสด้วยกุญแจสารธารณะจะไม่สามาตรรู้ได้ว่าข้อมูลที่ถูกส่งมานั้นเป็นข้อมูลที่ผู้ส่งตั้งใจส่งมาจริงๆหรือไม่ หรือจะถูกปลอมแปลงไหม เพราะว่าใครๆ ก็สามารถเข้าถึง public key ได้และสามารถเข้ารหัสข้อมูลใดๆ ก็ได้ จึงต้องทำการตรวจสอบข้อมูลว่าถูกส่งมาจากต้นทางจริงๆ หรือไม่ด้วยการตรวจสอบลายเซ็นดิจิทัล (digital signature verification) โดยผู้ส่งสารจะเซ็นข้อมูลด้วย private key ของตัวเอง และสร้างคู่ public key ขึ้นมาเพื่อให้ผู้รับสารสามารถ verify ได้ว่าข้อมูลถูกส่งออกมาจากผู้รับสารนั้นจริงๆ และไม่ได้ถูกปลอมแปลง และในการรักษาความลับ digital signature จะถูกสร้างจาก hash ของข้อมูลที่ถูกเข้ารหัสแล้วแทน  อย่างไรก็ตามกุญแจทั้งสองสามารถสร้างได้ด้วยตัวเองจึงทำให้ต้องมีการตรวจสอบและรับรองกุญแจสารณะด้วยคนกลางอีกส่วนหนึ่ง

ในตัวอย่งานี้จะแสดงวิธีการเข้ารหัสและถอดรหัสด้วยการเข้ารหัสแบบอสมมาตร การเซ็นด้วย digital signature ด้วยการเข้ารหัสแบบอสมมาตร แต่จะไม่แสดงวิธีการสร้างและรับรองกุญแจสาธารณะด้วยผู้ออกใบรับรองอิเล็กทรอนิกส์ (certificate authority: CA) ในตัวอย่างนี้จะมีการจำลองสถานะการณ์รับส่งข้อมูลระหว่าง Mr.A และ Mr.B โดยจะมีเงื่อนไขดังนี้

### Scenario:
* Mr.A ต้องการส่งข้อความเข้ารหัสไปให้ Mr.B
* การสื่อสารระหว่าง Mr.A และ Mr.B ต้องเข้ารหัสและถอดรหัสด้วย Asymmetric Keys
* Mr.B ต้องการ verify ผู้ส่งและข้อความว่าถูกต้องหรือไม่ด้วย MD5 Digital Signature
* แลกเปลี่ยน Key และ Verify กันสองคนโดยไม่มี CA และรับส่งผ่านทาง file

### Mr.B

Mr.B สร้าง private key สำหรับการสร้าง public-private key pair

In [1]:
openssl genrsa -out mr_b.key 512

Generating RSA private key, 512 bit long modulus (2 primes)
.+++++++++++++++++++++++++++
..+++++++++++++++++++++++++++
e is 65537 (0x010001)


Mr.B สร้าง public key เพื่อส่งให้ Mr.A ใช้ encrypt message

In [2]:
openssl rsa -in mr_b.key -pubout -out mr_b.pub.key.pem

writing RSA key


### Mr.A

Mr.A ต้องการส่ง message
```
"Hello B, I'am A."
```
ไปหา Mr.B

In [3]:
echo "Hello B, I'am A." > a2b_message.txt

Mr.A encrypt message เพื่อทำการส่งให้ Mr.B

In [4]:
openssl rsautl -encrypt -pubin -inkey mr_b.pub.key.pem -in a2b_message.txt -out encrypted_a2b_message.txt

ข้อมูลที่ถูก encrypted message

In [5]:
cat encrypted_a2b_message.txt

k���1���8�~_�)��%,l|�7F�gkٹZ Op�

Mr.A สร้าง private key สำหรับการสร้าง public-private key pair

In [6]:
openssl genrsa -out mr_a.key 512

Generating RSA private key, 512 bit long modulus (2 primes)
............+++++++++++++++++++++++++++
....+++++++++++++++++++++++++++
e is 65537 (0x010001)


Mr.A สร้าง public key เพื่อส่งให้ Mr.B validate signature ได้

In [7]:
openssl rsa -in mr_a.key -pubout -out mr_a.pub.key.pem

writing RSA key


Sign document เพื่อสร้าง digital signature

In [8]:
openssl dgst -md5 -sign mr_a.key -out digital_signature.md5 encrypted_a2b_message.txt

digital signature

In [9]:
cat digital_signature.md5

��K��T��[OΚg۳y��s�p1ϧ2!��g�p��G�5}��2��ERG�T׳��qy��UU

Mr.A ส่ง public key, digital signature และ encrypted message ให้ Mr.B

### Mr.B

รับ public key, digital signature และ encrypted message

Verify document ด้วย digital signature และ public key

In [10]:
openssl dgst -md5 -verify mr_a.pub.key.pem -signature digital_signature.md5 encrypted_a2b_message.txt

Verified OK


Trying to decrypt message

In [11]:
openssl rsautl -decrypt -inkey mr_b.key -in encrypted_a2b_message.txt -out decrypted_a2b_message.txt
cat decrypted_a2b_message.txt

Hello B, I'am A.


## Conclusion

- ใช้ Public-Private Key 2 คู่ คู่หนึ่งใช้เพื่อ encrypt-decrypt message และอีกคู่หนึ่งใช้เพื่อ sign-verify message
- public key จะถูกใช้เพื่อ encrypt เสมอ เนื่อจาก แม้ว่า public key จะหลุดสู่สาธารณะ ก็สามารถใช้ทำได้เพียงเข้ารหัสเท่านั้น เนื่องจากคู่ private key จะอยู่ที่ receiver เสมอ
- private key ที่ถูกใช้ sign message ก็เช่นกัน โดย private key จะอยู่ที่ sender เสมอ และ public key ที่ถูกส่งให้ก็ทำได้เพียงใช้ verify ท่านั้น
- public key และ private key สามารถใช้ได้ทั้งเข้ารหัสและถอดรหัสได้ แต่ปกติ private key จะต้องไม่ถูกส่งผ่านตัวกลาง จะมีเฉพาะ public key เท่านั้นี่ถูกส่งผ่านตัวกลาง
- digital signature ก็ใช้ asymmetric key encryption เช่นเดียวกัน แต่ใช้เพื่อ verify เพื่อป้องกันการโจมตีแบบ man-in-the-middle attack โดยจะใช้เพื่อ validate message ว่าถูกส่งมาจาก sender จริงหรือไม่ (ใครก็สามารถใช้ public key ของ receiver เพื่อ encrypt ข้อมูลใดๆ ได้เช่นเดียวกัน)
- Public-Private Key ทั้ง 2 คู่ยังไม่มีความน่าเชื่อถือ เนื่องจากสามารถ generate ได้เองและไม่มีคนรับรองว่าถูก generate มาด้วย Mr.A / Mr.B จริงๆ
- ยังไม่สามารถ verify public key ได้ อาจจถูกสับเปลี่ยน public key ระหว่างทางได้

## References

* [Public-Private-Key Encryption and Digital Signature](https://www.researchgate.net/figure/Public-Private-Key-Encryption-and-Digital-Signatures_fig31_277736892)
* [SSL Client Authentication step-by-step](https://www.makethenmakeinstall.com/2014/05/ssl-client-authentication-step-by-step/)